<a href="https://colab.research.google.com/github/ShirishaReddyV/AI-Sandbox/blob/main/unsw_network_Intrusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mrwellsdavid/unsw-nb15")

print("Path to dataset files:", path)

In [ ]:
import os

print("Downloaded to:", path)
print("Files in directory:")
print(os.listdir(path))


In [ ]:
import pandas as pd
import os

train_path = os.path.join(path, "UNSW_NB15_training-set.csv")
test_path = os.path.join(path, "UNSW_NB15_testing-set.csv")

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Combine
df = pd.concat([df_train, df_test], ignore_index=True)
print("Dataset shape:", df.shape)
print(df['label'].value_counts())
print(df['attack_cat'].value_counts(dropna=False))  # Shows attack categories


In [ ]:
!pip install dash plotly



In [ ]:
import dash
from dash import dcc, html
import plotly.express as px

app = dash.Dash(__name__)

fig = px.histogram(df, x='attack_cat', title="Attack Category Distribution")

app.layout = html.Div([
    html.H1("UNSW-NB15 Cybersecurity Dashboard"),
    dcc.Graph(figure=fig),
    dcc.Dropdown(
        options=[{'label': col, 'value': col} for col in df.columns if df[col].dtype != 'object'],
        value='dur',
        id='feature-dropdown'
    ),
    dcc.Graph(id='feature-hist')
])

@app.callback(
    dash.dependencies.Output('feature-hist', 'figure'),
    [dash.dependencies.Input('feature-dropdown', 'value')]
)
def update_hist(selected_feature):
    return px.histogram(df, x=selected_feature, color='attack_cat')

app.run(debug=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# One-hot encode categorical features
X = df.drop(columns=['label', 'attack_cat'])  # features
X = pd.get_dummies(X, columns=['proto', 'service', 'state'], dummy_na=False)

y_binary = df['label']                        # 0 = Normal, 1 = Attack
y_multi = df['attack_cat']                    # For multi-class

X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Binary Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:
# Install these in Colab or local
# !pip install dash plotly pandas

import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import os

# Load Dataset
path = "/kaggle/input/unsw-nb15"
df_train = pd.read_csv(os.path.join(path, "UNSW_NB15_training-set.csv"))
df_test = pd.read_csv(os.path.join(path, "UNSW_NB15_testing-set.csv"))
df = pd.concat([df_train, df_test], ignore_index=True)

# Fill NaNs
df.fillna("Unknown", inplace=True)

# Dash App
app = dash.Dash(__name__)
app.title = "UNSW-NB15 Cybersecurity Dashboard"

# Main Layout
app.layout = html.Div(style={
    'fontFamily': 'Arial',
    'backgroundColor': '#f8f9fa',
    'padding': '20px'
}, children=[

    html.H1("🔐 UNSW-NB15 Intrusion Detection Dashboard", style={
        'textAlign': 'center',
        'color': '#2c3e50'
    }),

    html.P("Explore attack types, protocols, and feature insights from the UNSW-NB15 dataset.", style={
        'textAlign': 'center',
        'color': '#34495e'
    }),

    html.Div([
        html.Label("🎯 Select Attack Category:", style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='attack-dropdown',
            options=[{'label': cat, 'value': cat} for cat in df['attack_cat'].unique()],
            value='Normal',
            clearable=False,
            style={'width': '300px'}
        ),
    ], style={'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='attack-distribution'),
        dcc.Graph(id='protocol-duration-heatmap'),
    ], style={'display': 'flex', 'gap': '20px'}),

    html.Div([
        html.Label("📈 Select Feature to Explore:", style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='feature-dropdown',
            options=[{'label': col, 'value': col} for col in df.columns if df[col].dtype in ['int64', 'float64']],
            value='dur',
            style={'width': '300px'}
        ),
        dcc.Graph(id='feature-graph')
    ], style={'marginTop': '40px'})
])

# Callbacks
@app.callback(
    Output('attack-distribution', 'figure'),
    Input('attack-dropdown', 'value')
)
def update_attack_chart(selected_cat):
    filtered = df[df['attack_cat'] == selected_cat]
    fig = px.histogram(filtered, x='proto', title=f"Protocol Usage for '{selected_cat}' Attacks", color='proto')
    fig.update_layout(template='plotly_white')
    return fig

@app.callback(
    Output('protocol-duration-heatmap', 'figure'),
    Input('attack-dropdown', 'value')
)
def update_heatmap(selected_cat):
    filtered = df[df['attack_cat'] == selected_cat]
    pivot = filtered.pivot_table(index='proto', columns='service', values='dur', aggfunc='mean').fillna(0)
    fig = px.imshow(pivot, text_auto=True, title=f"Avg Duration by Protocol & Service ({selected_cat})")
    fig.update_layout(template='plotly_white')
    return fig

@app.callback(
    Output('feature-graph', 'figure'),
    Input('feature-dropdown', 'value')
)
def update_feature_graph(feature_name):
    fig = px.box(df, x='attack_cat', y=feature_name, color='attack_cat',
                 title=f"📊 Distribution of '{feature_name}' by Attack Category")
    fig.update_layout(template='plotly_white')
    return fig

# Run app (for Colab/Notebook use JupyterDash instead)
app.run(debug=False, use_reloader=False)